In [1]:
from IPython import get_ipython
from IPython.display import display

# Installer les dépendances nécessaires
!pip install -U langgraph langsmith langchain_anthropic
!pip install -U openai
!pip install -U transformers huggingface_hub
!pip install torch

import os

# Charger le token Hugging Face depuis .env
from google.colab import files
uploaded = files.upload()  # Téléchargez votre fichier .env

from dotenv import load_dotenv
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM  # Import manquant

load_dotenv(".env")  # Charge votre token
token = os.getenv("HUGGINGFACEHUB_API_TOKEN")
print("✅ Token chargé :", token[:10] + "...")

# Charger le modèle Mistral
model_name = "mistralai/Mistral-7B-Instruct-v0.1"

# Charger le tokenizer et le modèle avec float16
tokenizer = AutoTokenizer.from_pretrained(model_name, token=token)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    token=token,
    torch_dtype=torch.float16,  # Forcer FP16 pour réduire la mémoire
    device_map="auto"  # GPU si disponible
)

# Vérifier le chargement du modèle
print("✅ Modèle chargé sur :", model.device)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.8/384.8 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.0/308.0 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 16.3 MB/s eta 0:00:00
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.4.24
    Uninstalling langsmith-0.4.24:
      Successfully uninstalled langsmith-0.4.24
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 951.0/951.0 kB 31.9 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.106.1
    Uninstalling openai-1.106.1:
      Successfully uninstalled openai-1.106.1


Saving .env to .env
✅ Token chargé : hf_KWIvcKW...


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

✅ Modèle chargé sur : cuda:0


In [2]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage
from transformers import AutoTokenizer
import textwrap
import random

prompts = [
    "Quels sont les principaux avantages de l'IA pour la société?",
    "Quels sont les dangers potentiels de l'IA?",
    "Comment l'IA peut-elle être utilisée pour résoudre des problèmes mondiaux?",
    "L'IA est-elle une menace pour l'emploi?",
    "Comment garantir une utilisation éthique de l'IA?",
]

# Définition de l'état du débat
class DebateState(TypedDict):
    messages: Annotated[list, add_messages]  # Liste des messages échangés

# Création du graphe
graph_builder = StateGraph(DebateState)

def agent1(state: DebateState):
    """L'agent 1 donne son point de vue"""
    last_message = state["messages"][-1].content
    # Extraire le prompt aléatoire du dernier message
    last_prompt = last_message.split(" ")[-1]

    # Formuler l'input pour le modèle en fonction du prompt
    input_text = f"## Argument en faveur de l'IA concernant: {last_prompt}\n"  # Début de l'argument

    # ... (Vous pouvez ajouter des instructions spécifiques ici)
    input_text += "Veuillez développer un argument en faveur des bénéfices de l'IA en vous concentrant sur ce point."

    # Définir 'inputs'
    inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=150)

    response_text = tokenizer.decode(output[0], skip_special_tokens=True)
    response_text = textwrap.fill(response_text, width=80)
    return {"messages": state["messages"] + [AIMessage(content=response_text, type='ai')]}

def agent2(state: DebateState):
    """L'agent 2 réagit à l'argument de l'agent 1"""
    last_message = state["messages"][-1].content
    last_prompt = last_message.split(" ")[-1]  # Supposant que le prompt est le dernier mot

    # Formuler l'input pour le modèle en fonction du prompt
    input_text = f"## Argument contre l'IA concernant: {last_prompt}\n"

    # ... (Vous pouvez ajouter des instructions spécifiques ici)
    input_text += "Veuillez développer un argument contre les dangers de l'IA en vous concentrant sur ce point."

    # Définir 'inputs'
    inputs = tokenizer(input_text, return_tensors="pt").to(model.device)


    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=150, temperature=0.8)

    response_text = tokenizer.decode(output[0], skip_special_tokens=True)

    # Ajuster le retour à la ligne du texte
    response_text = textwrap.fill(response_text, width=80)  # Ajuster la largeur selon vos besoins
    # Retourner un AIMessage
    return {"messages": state["messages"] + [AIMessage(content=response_text, type='ai')]}

# Ajouter les agents dans le graphe
graph_builder.add_node("agent1", agent1)
graph_builder.add_node("agent2", agent2)

# Lancement du débat par l'agent 1
graph_builder.add_edge(START, "agent1")
# L'agent 1 passe la main à l'agent 2
graph_builder.add_edge("agent1", "agent2")
# L'agent 2 renvoie à l'agent 1 pour continuer le débat
graph_builder.add_edge("agent2", "agent1")

In [3]:
# Compiler le graphe
debate_graph = graph_builder.compile()

# Fonction pour simuler le débat
def run_debate(initial_prompt, num_turns=5):
    # Initialiser avec HumanMessage
    state = {"messages": [HumanMessage(content=initial_prompt)]}

    for i, event in enumerate(debate_graph.stream(state)):
       # Choisir un prompt aléatoire pour ce tour
        current_prompt = random.choice(prompts)

        # Ajouter le prompt au message du tour actuel
        state['messages'][-1].content += " " + current_prompt # on ajoute le prompt current au dernier prompt existant

        for value in event.values():
            # Convertir le dernier message en AIMessage ou HumanMessage
            last_message = value['messages'][-1]
            # Accéder au type du message avec .type
            if last_message.type in ("agent1", "agent2"):
                message = AIMessage(content=last_message.content)
            else:
                message = HumanMessage(content=last_message.content)

            print(f"🗣️ Tour {i+1} - {message.type} : {message.content}")

        if i >= num_turns - 1:
            break


# Lancer le débat avec un sujet donné
run_debate("L'intelligence artificielle est-elle bénéfique ou dangereuse ?")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


🗣️ Tour 1 - human : ## Argument en faveur de l'IA concernant: ? Veuillez développer un argument en
faveur des bénéfices de l'IA en vous concentrant sur ce point.  Argument en
faveur de l'IA concernant: l'IA peut améliorer les performances des entreprises
L'IA peut améliorer les performances des entreprises en plusieurs façons.
Premièrement, l'IA peut automatiser des tâches répétitives et monotones, ce qui
peut libérer les employés de ces tâches et les permettre de se concentrer sur
des tâches plus créatives et plus intéressantes.  Deuxièmement, l'IA peut
analyser des données et fournir des insights et des recommandations à
l'entreprise. Ces insights


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


🗣️ Tour 2 - human : ## Argument contre l'IA concernant: insights Veuillez développer un argument
contre les dangers de l'IA en vous concentrant sur ce point.  L'IA est une
technologie qui a le potentiel de transformer notre monde en une société où les
humains et les machines coexistent. Cependant, il est important de reconnaître
que l'IA est une technologie qui peut être utilisée pour des fins malveillantes.
Un des dangers de l'IA est qu'elle peut être utilisée pour des fins de
surveillance et de contrôle. Par exemple, les gouvernements peuvent utiliser
l'IA pour surveiller les citoyens et contrôler leur activité. Cette surveillance
peut être utilisée pour des fins de contrôle et de répression, et peut être
utilisée pour des fins de


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


🗣️ Tour 3 - human : ## Argument en faveur de l'IA concernant: de Veuillez développer un argument en
faveur des bénéfices de l'IA en vous concentrant sur ce point.  Deux points sont
à considérer lors de l'analyse des bénéfices de l'IA:  1. L'IA peut améliorer
les processus de décision et de prise d'action. 2. L'IA peut augmenter la
productivité et la qualité des services.  L'IA peut améliorer les processus de
décision et de prise d'action en fournissant des informations et des conseils à
l'aide d'algorithmes et de modèles de prédiction. Par exemple, dans le domaine
de la santé, l'IA peut aider les médecins


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


🗣️ Tour 4 - human : ## Argument contre l'IA concernant: médecins Veuillez développer un argument
contre les dangers de l'IA en vous concentrant sur ce point.  L'IA est une
technologie qui a le potentiel de révolutionner le monde entier. Elle peut être
utilisée pour des applications positives telles que la médecine, la recherche et
l'éducation. Cependant, il est important de reconnaître les dangers de l'IA et
de prendre des mesures pour les éviter.  L'IA dans la médecine est une
application qui a le potentiel de changer la façon dont nous traitons les
maladies. Elle peut être utilisée pour des analyses médicales, des diagnostics
et des traitements. Cependant, il est important de reconnaître les dangers de
🗣️ Tour 5 - human : ## Argument en faveur de l'IA concernant: de Veuillez développer un argument en
faveur des bénéfices de l'IA en vous concentrant sur ce point.  Deux points sont
à considérer lors de l'analyse des bénéfices de l'IA:  1. L'IA peut améliorer
les processus de décision e